In [1]:
import datetime
from dateutil.parser import parse as parse_datetime
import re
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import smtplib
from email.mime.text import MIMEText
from pytz import UTC, timezone
import spacy
import nltk
from nltk.tokenize import word_tokenize

In [2]:
nlp = spacy.load("en_core_web_sm")

api_scope = ['https://www.googleapis.com/auth/calendar']
rec_mail = "jasmakhija1234@gmail.com"
conf_mail = "religious7903@gmail.com"
app_pass = "amltccltzhlhwiww"
mail_regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
ist = timezone('Asia/Kolkata')
st_hour = 9
end_hour = 16 

In [3]:
def check():
    creds = None
    try:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', api_scope)
        creds = flow.run_local_server(port=3000)
        print("succ auth")
    except FileNotFoundError:
        print("check folder for file cred")
        return None
    except Exception as e:
        print(f" auth error is : {e}")
        return None
    return build('calendar', 'v3', credentials=creds)

In [4]:
def check_slot_availability(gservice_obj, start_time, end_time):
    if gservice_obj is None:
        return False
    events_result = gservice_obj.events().list(
        calendarId='primary', timeMin=start_time.isoformat(), timeMax=end_time.isoformat(),
        singleEvents=True, orderBy='startTime').execute()
    return len(events_result.get('items', [])) == 0

In [5]:
def parse_date_time(text):
    text = text.lower().strip()
    today = datetime.datetime.now(ist).date()
    now_utc = datetime.datetime.now(UTC)
    tokens = word_tokenize(text)
    doc = nlp(text)
    
    target_date = None
    target_time = None
    
    weekdays = {
        'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3, 'friday': 4, 'saturday': 5, 'sunday': 6,
        'mon': 0, 'tue': 1, 'wed': 2, 'thu': 3, 'fri': 4, 'sat': 5, 'sun': 6
    }
    
    date_time_match = re.search(r'(\d{1,2})\s*([a-z]+)\s*(\d{1,2})(am|pm)?', text)
    if date_time_match:
        day = int(date_time_match.group(1))
        month_str = date_time_match.group(2)[:3]
        hour = int(date_time_match.group(3))
        period = date_time_match.group(4)
        
        months = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}
        
        if month_str in months:
            month = months[month_str]
            year = today.year if datetime.date(today.year, month, day) >= today else today.year + 1
            if period == "pm" and hour < 12:
                hour = hour + 12
            elif period == "am" and hour == 12:
                hour = 0
            target_date = datetime.date(year, month, day)
            target_time = datetime.time(hour, 0)
    
    if not target_date:
        for ent in doc.ents:
            if ent.label_ == "DATE":
                try:
                    dt = parse_datetime(ent.text, fuzzy=True, default=datetime.datetime.combine(today, datetime.time(0, 0)))
                    target_date = dt.date()
                except ValueError:
                    continue
            elif ent.label_ == "TIME":
                try:
                    dt = parse_datetime(ent.text, fuzzy=True, default=datetime.datetime.combine(today, datetime.time(0, 0)))
                    target_time = dt.time()
                except ValueError:
                    continue
    
    for i, token in enumerate(doc):
        if token.text in weekdays:
            days_ahead = weekdays[token.text] - today.weekday()
            if days_ahead <= 0:
                days_ahead += 7
            target_date = today + datetime.timedelta(days=days_ahead)
        
        if token.text == "today":
            target_date = today
        elif token.text == "tomorrow":
            target_date = today + datetime.timedelta(days=1)
        
        if token.text == "morning":
            target_time = datetime.time(10, 0)
        elif token.text == "afternoon":
            target_time = datetime.time(14, 0)
        
        if token.pos_ == "NUM" and token.head.text in ["am", "pm"]:
            hour = int(token.text)
            if token.head.text == "pm" and hour < 12:
                hour += 12
            elif token.head.text == "am" and hour == 12:
                hour = 0
            target_time = datetime.time(hour, 0)
    
    if not target_time:
        time_match = re.search(r'(\d{1,2})(?::(\d{2}))?\s*(am|pm)?', text)
        if time_match:
            hour = int(time_match.group(1))
            minute = int(time_match.group(2)) if time_match.group(2) else 0
            period = time_match.group(3)
            if period == "pm" and hour < 12:
                hour += 12
            elif period == "am" and hour == 12:
                hour = 0
            if 0 <= hour < 24 and 0 <= minute < 60:
                target_time = datetime.time(hour, minute)
    
    if not target_date:
        target_date = today
    if not target_time:
        target_time = datetime.time(st_hour, 0)
    
    local_dt = ist.localize(datetime.datetime.combine(target_date, target_time))
    utc_dt = local_dt.astimezone(UTC)
    
    if (local_dt.hour < st_hour or local_dt.hour > end_hour or
        utc_dt < now_utc):
        return None
    
    return utc_dt

In [6]:
def get_valid_availability():
    while True:
        availability = input("When are you available? /n write in format such as '20 march 2pm', 'tomorrow 9am', 'monday 4pm', 'tue 3pm': ")
        target_datetime = parse_date_time(availability)
        
        if target_datetime:
            end_time = target_datetime + datetime.timedelta(hours=1)
            return target_datetime
        else:
            print("Invalid input, time outside 9 AM to 4 PM IST, or in the past. Please try again.")

In [7]:
def schedule_meeting(gservice_obj, start_time, end_time, candidate_email):
    if gservice_obj is None:
        return None
    meeting_event = {
        'summary': 'intervieww',
        'start': {'dateTime': start_time.isoformat(), 'timeZone': 'UTC'},
        'end': {'dateTime': end_time.isoformat(), 'timeZone': 'UTC'},
        'attendees': [{'email': rec_mail}, {'email': candidate_email}],
        'reminders': {'useDefault': False, 'overrides': [{'method': 'email', 'minutes': 30}, {'method': 'popup', 'minutes': 10}]},
        'conferenceData': {'createRequest': {'requestId': f"{start_time.strftime('%Y%m%d%H%M%S')}", 'conferenceSolutionKey': {'type': 'hangoutsMeet'}}}
    }
    
    try:
        return gservice_obj.events().insert(calendarId='primary', body=meeting_event, conferenceDataVersion=1, sendUpdates='all').execute()
    except HttpError as error:
        print(f"An error occurred while creating the meeting: {error}")
        return None

In [8]:
def send_email(candidate_email, start_time, end_time, meet_link):
    local_start = start_time.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')
    local_end = end_time.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')
    
    msg_body = f"Interview scheduled from {local_start} to {local_end}\nJoin the meeting here: {meet_link}"
    msg = MIMEText(msg_body)
    msg['Subject'] = 'interview confirmation'
    msg['From'] = conf_mail
    msg['To'] = candidate_email
    
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(conf_mail, app_pass)
            server.send_message(msg)
        return True
    except smtplib.SMTPException as e:
        print(f"SMTP Error: {e}")
        return False


In [9]:
def get_available_slots_before(gservice_obj, target_datetime, duration=datetime.timedelta(hours=1)):
    target_date = target_datetime.astimezone(ist).date()
    start_of_day = ist.localize(datetime.datetime.combine(target_date, datetime.time(st_hour, 0)))
    end_limit = min(target_datetime, ist.localize(datetime.datetime.combine(target_date, datetime.time(end_hour, 0))))
    
    available_slots = []
    current_start = start_of_day
    
    while current_start + duration <= end_limit:
        current_end = current_start + duration
        if check_slot_availability(gservice_obj, current_start, current_end):
            available_slots.append((current_start, current_end))
        current_start += datetime.timedelta(hours=1)
    
    return available_slots

In [10]:
def get_available_slots_after(gservice_obj, target_datetime, duration=datetime.timedelta(hours=1)):
    target_date = target_datetime.astimezone(ist).date()
    start_limit = target_datetime
    end_of_day = ist.localize(datetime.datetime.combine(target_date, datetime.time(end_hour, 0)))
    
    available_slots = []
    current_start = start_limit
    
    while current_start + duration <= end_of_day:
        current_end = current_start + duration
        if check_slot_availability(gservice_obj, current_start, current_end):
            available_slots.append((current_start, current_end))
        current_start += datetime.timedelta(hours=1)
    
    return available_slots

In [11]:
def get_available_slots_between(gservice_obj, target_date, duration=datetime.timedelta(hours=1)):
    start_of_day = ist.localize(datetime.datetime.combine(target_date, datetime.time(st_hour, 0)))
    end_of_day = ist.localize(datetime.datetime.combine(target_date, datetime.time(end_hour, 0)))
    
    available_slots = []
    current_start = start_of_day
    
    while current_start + duration <= end_of_day:
        current_end = current_start + duration
        if check_slot_availability(gservice_obj, current_start, current_end):
            available_slots.append((current_start, current_end))
        current_start += datetime.timedelta(hours=1)
    
    return available_slots

In [12]:
def main():
    gservice_obj = check()
    if gservice_obj is None:
        print("auth failed with google cal")
        return
    
    while True:
        print("\nAI scheduling bot")
        #print("enter date and time")
        print(" enter in format like: '20 march 2pm', 'today 4pm', 'monday 4pm', 'tue 3pm'")
        
        candidate_email = input("enter your email: ").strip()
        if not re.match(mail_regex, candidate_email):
            print("invalid email format. pls. enter a valid email address")
            continue
        
        target_datetime = get_valid_availability()
        if not target_datetime:
            continue
        
        meeting_duration = datetime.timedelta(hours=1)
        requested_end = target_datetime + meeting_duration
        
        if check_slot_availability(gservice_obj, target_datetime, requested_end):
            start_time = target_datetime
            end_time = requested_end
        else:
            print(f"Sorry the requested time /n"
                  f"{target_datetime.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')} is not available.")
            
            
            target_date = target_datetime.astimezone(ist).date()
            slots_before = get_available_slots_before(gservice_obj, target_datetime)
            slots_after = get_available_slots_after(gservice_obj, target_datetime)
            slots_all = get_available_slots_between(gservice_obj, target_date)
            
            if not slots_all:
                print(f"no available slots on {target_date.strftime('%Y-%m-%d')} between {st_hour}:00 and {end_hour}:00 IST")
                continue
            
            print("\n available slots for the day ")
            for slot_start, slot_end in slots_all:
                print(f"{slot_start.astimezone(ist).strftime('%I:%M %p')} - "
                      f"{slot_end.astimezone(ist).strftime('%I:%M %p')}")
            
            choice = input("\n would you like to book one of these slots write y for yes (y/n): ").lower()
            if choice != 'y':
                continue
                
            while True:
                new_time = input("enter the start time you'd like (e.g., '2:00 pm'): ").strip()
                try:
                    parsed_time = parse_datetime(new_time, default=datetime.datetime.combine(target_date, datetime.time(0, 0)))
                    new_start = ist.localize(parsed_time)
                    new_end = new_start + meeting_duration
                    
                    if (new_start.hour < st_hour or new_end.hour > end_hour or
                        new_start < datetime.datetime.now(ist)):
                        print(" time outside working hours or in the past \n pl try again")
                        continue
                    
                    if check_slot_availability(gservice_obj, new_start, new_end):
                        start_time = new_start
                        end_time = new_end
                        break
                    else:
                        print("this slot is not available pl choose another time")
                except ValueError:
                    print("invalid time format. pl try again.")
        
        if not check_slot_availability(gservice_obj, start_time, end_time):
            print("Slot unavailable")
            continue
        
        meeting_event = schedule_meeting(gservice_obj, start_time, end_time, candidate_email)
        
        if meeting_event and 'id' in meeting_event:
            meet_link = meeting_event.get('hangoutLink', 'no google meet link generated')
            print(f"meeting scheduled successfully "
                  f"{start_time.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')} to "
                  f"{end_time.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')}")
            print(f"google meet link {meet_link}")
            
            email_sent = send_email(candidate_email, start_time, end_time, meet_link)
            if email_sent:
                print("conf. email sent with meet link")
            else:
                print("failed to send confirmation email")
        else:
            print("failed to schedule the meeting in google galendar")
        
        if input("schedule another meet?(y/n): ").lower() != 'y':
            break

In [13]:
main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=998403459185-kma8aj7rp8skb481irbr751smpnctv7b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=ZzjZEgRsGvEinILFIRzyhop3BtmB9B&access_type=offline
succ auth

AI scheduling bot
 enter in format like: '20 march 2pm', 'today 4pm', 'monday 4pm', 'tue 3pm'


enter your email:  jasmakhija1234


invalid email format. pls. enter a valid email address

AI scheduling bot
 enter in format like: '20 march 2pm', 'today 4pm', 'monday 4pm', 'tue 3pm'


enter your email:  jasmakhija1234@gmail.com
When are you available? /n write in format such as '20 march 2pm', 'tomorrow 9am', 'monday 4pm', 'tue 3pm':  tomorrow 2pm


Sorry the requested time /n2025-03-20 02:00 PM IST is not available.

 available slots for the day 
10:00 AM - 11:00 AM
11:00 AM - 12:00 PM
01:00 PM - 02:00 PM
03:00 PM - 04:00 PM


...

enter your email:  jasmakhija1234@gmail.com
When are you available? /n write in format such as '20 march 2pm', 'tomorrow 9am', 'monday 4pm', 'tue 3pm':  today 6pm


Invalid input, time outside 9 AM to 4 PM IST, or in the past. Please try again.


When are you available? /n write in format such as '20 march 2pm', 'tomorrow 9am', 'monday 4pm', 'tue 3pm':  friday 9am


meeting scheduled successfully 2025-03-21 09:00 AM IST to 2025-03-21 10:00 AM IST
google meet link https://meet.google.com/itt-ypws-nin
conf. email sent with meet link


schedule another meet?(y/n):  n
